# DL Mini Project

## Submitted By: Ishwant Singh Bhayana(isb5064), Prajna Ravindra Nayak(pn2224), Parth Mehta(pjm9767) 

# Initial Setup


In [1]:
!pip install torchinfo

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

import torchvision
import torchvision.transforms as transforms

import pandas as pd
import numpy as np

from torchinfo import summary

## Training data from CIFAR10

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
best_acc = 0  # start with 0, will update as better acc is achieved
start_epoch = 0

transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

trainset = torchvision.datasets.CIFAR10(
    root='./data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=128, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=100, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')

# Reference code: https://github.com/kuangliu/pytorch-cifar/blob/master/models/resnet.py

==> Preparing data..


100%|██████████| 170498071/170498071 [00:13<00:00, 12782749.88it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


## Model

In [4]:
class Block(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(Block, self).__init__()
        self.conv1 = nn.Conv2d(
            in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

class ResNetModel(nn.Module):
    def __init__(self, block, num_blocks, strides=[1,2,2,2], num_classes=10):
        super(ResNetModel, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=strides[0])
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=strides[1])
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=strides[2])
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=strides[3])
        self.linear = nn.Linear(512*block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        if num_blocks > 0:
            strides = [stride] + [1]*(num_blocks-1)
            layers = []
            for stride in strides:
                layers.append(block(self.in_planes, planes, stride))
                self.in_planes = planes * block.expansion
            return nn.Sequential(*layers)
        else:
            # Use a single convolution layer instead of a residual block when num_blocks is 0
            layer =  nn.Sequential(nn.Conv2d(self.in_planes, planes, kernel_size=1,
                                stride=stride, padding=1, bias=False),
                          nn.BatchNorm2d(planes))
            self.in_planes = planes * block.expansion
            return layer


    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

# Reference code: https://github.com/kuangliu/pytorch-cifar/blob/master/models/resnet.py

## Training

In [5]:
learning_rate = 0.1

# Train
def train_one_epoch(net, optimizer, criterion):
    
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, labels) in enumerate(trainloader):

        optimizer.zero_grad()

        inputs, labels = inputs.to(device), labels.to(device)
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()

        optimizer.step()

        train_loss += loss.item()
        
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

        
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {100.*correct/total:.2f}%')

    return train_loss, 100.*correct/total


# Test
def test(net, optimizer, criterion):

    global best_acc
    net.eval()
    test_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():

        for batch_idx, (inputs, labels) in enumerate(testloader):

            inputs, labels = inputs.to(device), labels.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, labels)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

        print(f'\tTest Loss: {test_loss:.3f} | Test Acc: {100.*correct/total:.2f}%')

        return test_loss, 100.*correct/total

    #save the checkpoint
    acc = 100.*correct/total
    if acc > best_acc: #when curr acc is the best yet
        state = {
            'net': net.state_dict(),
            'acc': acc,
            'epoch': epoch,
        }
        if not os.path.isdir('checkpoint'):
            os.mkdir('checkpoint')
        torch.save(state, './checkpoint/ckpt.pth')
        best_acc = acc


# Reference code: https://github.com/kuangliu/pytorch-cifar/blob/master/models/resnet.py

In [6]:
EPOCHS = 50

def train(net, learning_rate=0.1):

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=learning_rate,
                        momentum=0.9, weight_decay=5e-4)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)

    train_accuracy_history = []
    train_loss_history = []
    test_accuracy_history = []
    test_loss_history = []

    for epoch in range(EPOCHS):
      
        print('\nEpoch: %d' % epoch)
        train_loss, train_acc = train_one_epoch(net, optimizer, criterion)
        test_loss, test_acc = test(net, optimizer, criterion)

        train_accuracy_history.append(train_acc)

        train_loss_history.append(train_loss)
        test_accuracy_history.append(test_acc)
        test_loss_history.append(test_loss)

        scheduler.step()

    return {'train': (train_loss_history, train_accuracy_history), 'validation': (test_loss_history, test_accuracy_history)}

In [7]:
!mkdir ./results

In [8]:
def save_to_excel(data, table_name):

  df = pd.DataFrame(data=data)
  df = df.T

  df.to_excel(f'./results/{table_name}.xlsx')

def parse_results(results, phase, stat='accuracy'):

  stats = {}

  stat = 1 if stat == 'accuracy' else 0

  for key in results.keys():
    
      avg = np.average(results[key][phase][stat])
      max = np.max(results[key][phase][stat])
      stats[key] = {'average': avg, 'max': max}

  return stats

## Experimenting with parameter sweeps

In [ ]:
results = {}
for layer1_blocks in range(1, 5):
  for layer2_blocks in range(1, 4):
      for layer3_blocks in range(3):
          for layer4_blocks in range(2):
            num_blocks = [layer1_blocks, layer2_blocks, layer3_blocks, layer4_blocks]
            net = ResNetModel(Block, num_blocks)
            net = net.to(device)
            if device == 'cuda':
                net = torch.nn.DataParallel(net)
                cudnn.benchmark = True

            print(f'num_blocks={num_blocks}')

            model_summary = summary(net, (128, 3, 32, 32))
            n_params = model_summary.trainable_params

            if n_params > 5000000:
                print(f"Model has too many parameters ({n_params}), will skip")
                continue
            else:
                print(f"Will train model with {n_params}  parameters")

            result = train(net)
            results[f'{layer1_blocks}{layer2_blocks}{layer3_blocks}{layer4_blocks}'] = result
            
            save_to_excel(parse_results(results, 'validation'), 'WIP_block_num_results_val')
            save_to_excel(parse_results(results, 'train'), 'WIP_block_num_results_train')

save_to_excel(parse_results(results, 'validation'), 'block_num_results_val')
save_to_excel(parse_results(results, 'train'), 'block_num_results_train')

num_blocks=[1, 1, 0, 0]
Will train model with 476490  parameters

Epoch: 0


/usr/local/lib/python3.9/dist-packages/torchinfo/torchinfo.py:477: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  action_fn=lambda data: sys.getsizeof(data.storage()),
/usr/local/lib/python3.9/dist-packages/torch/storage.py:665: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return super().__sizeof__() + self.nbytes()


	Train Loss: 683.527 | Train Acc: 34.18%
	Test Loss: 158.107 | Test Acc: 41.91%

Epoch: 1
	Train Loss: 561.566 | Train Acc: 47.55%
	Test Loss: 139.102 | Test Acc: 49.33%

Epoch: 2
	Train Loss: 508.723 | Train Acc: 52.87%
	Test Loss: 130.675 | Test Acc: 53.05%

Epoch: 3
	Train Loss: 472.265 | Train Acc: 56.98%
	Test Loss: 121.175 | Test Acc: 57.22%

Epoch: 4
	Train Loss: 446.528 | Train Acc: 59.15%
	Test Loss: 144.332 | Test Acc: 51.33%

Epoch: 5
	Train Loss: 428.520 | Train Acc: 61.00%
	Test Loss: 103.931 | Test Acc: 63.07%

Epoch: 6
	Train Loss: 410.790 | Train Acc: 62.82%
	Test Loss: 127.526 | Test Acc: 56.23%

Epoch: 7
	Train Loss: 400.428 | Train Acc: 63.80%
	Test Loss: 116.243 | Test Acc: 59.32%

Epoch: 8
	Train Loss: 392.956 | Train Acc: 64.76%
	Test Loss: 145.597 | Test Acc: 52.73%

Epoch: 9
	Train Loss: 383.737 | Train Acc: 65.59%
	Test Loss: 127.416 | Test Acc: 57.66%

Epoch: 10
	Train Loss: 379.558 | Train Acc: 65.92%
	Test Loss: 120.578 | Test Acc: 59.80%

Epoch: 11
	Train L

/usr/local/lib/python3.9/dist-packages/torchinfo/torchinfo.py:477: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  action_fn=lambda data: sys.getsizeof(data.storage()),
/usr/local/lib/python3.9/dist-packages/torch/storage.py:665: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return super().__sizeof__() + self.nbytes()


	Train Loss: 598.101 | Train Acc: 43.72%
	Test Loss: 133.946 | Test Acc: 53.27%

Epoch: 1
	Train Loss: 428.216 | Train Acc: 60.75%
	Test Loss: 104.126 | Test Acc: 62.17%

Epoch: 2
	Train Loss: 353.731 | Train Acc: 67.92%
	Test Loss: 89.714 | Test Acc: 69.11%

Epoch: 3
	Train Loss: 306.321 | Train Acc: 72.39%
	Test Loss: 94.685 | Test Acc: 68.85%

Epoch: 4
	Train Loss: 276.106 | Train Acc: 75.70%
	Test Loss: 95.604 | Test Acc: 69.34%

Epoch: 5
	Train Loss: 257.245 | Train Acc: 77.18%
	Test Loss: 85.280 | Test Acc: 71.10%

Epoch: 6
	Train Loss: 244.435 | Train Acc: 78.41%
	Test Loss: 84.721 | Test Acc: 71.88%

Epoch: 7
	Train Loss: 235.258 | Train Acc: 79.12%
	Test Loss: 62.280 | Test Acc: 78.12%

Epoch: 8
	Train Loss: 230.589 | Train Acc: 79.41%
	Test Loss: 64.203 | Test Acc: 78.22%

Epoch: 9
	Train Loss: 224.092 | Train Acc: 80.23%
	Test Loss: 76.534 | Test Acc: 74.15%

Epoch: 10
	Train Loss: 216.944 | Train Acc: 81.00%
	Test Loss: 75.627 | Test Acc: 74.18%

Epoch: 11
	Train Loss: 213.

...

	Train Loss: 612.912 | Train Acc: 41.83%
	Test Loss: 128.139 | Test Acc: 54.27%

Epoch: 1
	Train Loss: 427.453 | Train Acc: 60.81%
	Test Loss: 107.843 | Test Acc: 62.14%

Epoch: 2
	Train Loss: 332.986 | Train Acc: 69.91%
	Test Loss: 87.448 | Test Acc: 70.67%

Epoch: 3
	Train Loss: 279.888 | Train Acc: 75.06%
	Test Loss: 104.718 | Test Acc: 68.66%

Epoch: 4
	Train Loss: 245.948 | Train Acc: 78.28%
	Test Loss: 64.820 | Test Acc: 78.16%

Epoch: 5
	Train Loss: 224.103 | Train Acc: 80.12%
	Test Loss: 66.321 | Test Acc: 77.89%

Epoch: 6
	Train Loss: 210.410 | Train Acc: 81.69%
	Test Loss: 103.771 | Test Acc: 69.31%

Epoch: 7
	Train Loss: 202.172 | Train Acc: 82.31%
	Test Loss: 67.518 | Test Acc: 77.49%

Epoch: 8
	Train Loss: 191.347 | Train Acc: 83.29%
	Test Loss: 54.636 | Test Acc: 81.34%

Epoch: 9
	Train Loss: 182.096 | Train Acc: 83.95%
	Test Loss: 67.097 | Test Acc: 78.43%

Epoch: 10
	Train Loss: 178.285 | Train Acc: 84.60%
	Test Loss: 77.268 | Test Acc: 76.32%

Epoch: 11
	Train Loss: 17

/usr/local/lib/python3.9/dist-packages/torchinfo/torchinfo.py:477: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  action_fn=lambda data: sys.getsizeof(data.storage()),
/usr/local/lib/python3.9/dist-packages/torch/storage.py:665: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return super().__sizeof__() + self.nbytes()


	Train Loss: 674.894 | Train Acc: 34.70%
	Test Loss: 155.408 | Test Acc: 41.42%

Epoch: 1
	Train Loss: 548.892 | Train Acc: 48.49%
	Test Loss: 148.025 | Test Acc: 47.95%

Epoch: 2
	Train Loss: 483.555 | Train Acc: 55.25%
	Test Loss: 132.071 | Test Acc: 53.35%

Epoch: 3
	Train Loss: 443.670 | Train Acc: 59.21%
	Test Loss: 138.342 | Test Acc: 52.14%

Epoch: 4
	Train Loss: 414.273 | Train Acc: 62.04%
	Test Loss: 107.252 | Test Acc: 61.57%

Epoch: 5
	Train Loss: 396.447 | Train Acc: 64.16%
	Test Loss: 125.538 | Test Acc: 57.30%

Epoch: 6
	Train Loss: 377.292 | Train Acc: 65.86%
	Test Loss: 111.377 | Test Acc: 61.33%

Epoch: 7
	Train Loss: 366.559 | Train Acc: 67.27%
	Test Loss: 118.327 | Test Acc: 58.77%

Epoch: 8
	Train Loss: 354.224 | Train Acc: 68.20%
	Test Loss: 115.464 | Test Acc: 61.04%

Epoch: 9
	Train Loss: 344.231 | Train Acc: 69.19%
	Test Loss: 108.264 | Test Acc: 63.88%

Epoch: 10
	Train Loss: 336.694 | Train Acc: 69.84%
	Test Loss: 105.465 | Test Acc: 64.31%

Epoch: 11
	Train L

/usr/local/lib/python3.9/dist-packages/torchinfo/torchinfo.py:477: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  action_fn=lambda data: sys.getsizeof(data.storage()),
/usr/local/lib/python3.9/dist-packages/torch/storage.py:665: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return super().__sizeof__() + self.nbytes()


	Train Loss: 588.607 | Train Acc: 44.69%
	Test Loss: 125.353 | Test Acc: 55.09%

Epoch: 1
	Train Loss: 419.162 | Train Acc: 61.51%
	Test Loss: 123.125 | Test Acc: 57.02%

Epoch: 2
	Train Loss: 346.780 | Train Acc: 68.66%
	Test Loss: 85.911 | Test Acc: 69.83%

Epoch: 3
	Train Loss: 297.298 | Train Acc: 73.14%
	Test Loss: 76.875 | Test Acc: 73.22%

Epoch: 4
	Train Loss: 267.175 | Train Acc: 76.20%
	Test Loss: 74.537 | Test Acc: 74.54%

Epoch: 5
	Train Loss: 244.757 | Train Acc: 78.39%
	Test Loss: 82.728 | Test Acc: 71.36%

Epoch: 6
	Train Loss: 233.753 | Train Acc: 79.43%
	Test Loss: 72.646 | Test Acc: 75.13%

Epoch: 7
	Train Loss: 224.240 | Train Acc: 80.30%
	Test Loss: 105.913 | Test Acc: 67.51%

Epoch: 8
	Train Loss: 218.020 | Train Acc: 80.83%
	Test Loss: 84.643 | Test Acc: 71.16%

Epoch: 9
	Train Loss: 210.811 | Train Acc: 81.37%
	Test Loss: 60.413 | Test Acc: 79.34%

Epoch: 10
	Train Loss: 206.059 | Train Acc: 81.82%
	Test Loss: 81.421 | Test Acc: 73.97%

Epoch: 11
	Train Loss: 198

## Experimenting with stride

In [ ]:
results = {}
for layer1stride in range(1, 4):
  for layer2stride in range(1, 4):
    for layer3stride in range(1, 4):
        # Best from previous sweep
        num_blocks=[2, 3, 2, 0]

        strides=[layer1stride, layer2stride, layer3stride, 2]

        net = ResNetModel(Block, num_blocks, strides=strides)
        net = net.to(device)
        if device == 'cuda':
            net = torch.nn.DataParallel(net)
            cudnn.benchmark = True
        print(f'strides={strides}')

        try:
            model_summary = summary(net, (128, 3, 32, 32))
            n_params = model_summary.trainable_params
            if n_params > 5000000:
                print(f"Model has too many parameters ({n_params}), will skip")
                continue
            else:
                print(f"Will train model with {n_params}  parameters")
        except:
            print(f"Invalid combination (strides={strides}), will skip. Most likely a dimensionality error.")
            continue

        result = train(net)
        results[f'strides:{layer1stride}{layer2stride}{layer3stride}'] = result

        save_to_excel(parse_results(results, 'validation'), 'WIP_stride_results_val')
        save_to_excel(parse_results(results, 'train'), 'WIP_stride_results_train')

save_to_excel(parse_results(results, 'validation'), 'stride_results_val')
save_to_excel(parse_results(results, 'train'), 'stride_results_train')

## Trying different learning rates

In [ ]:
results = {}
learning_rate = 0.02
while learning_rate < 0.2:

    # Best from previous sweep
    num_blocks=[2, 3, 2, 0]
    strides=[1,2,2,2]

    net = ResNetModel(Block, num_blocks, strides=strides)
    net = net.to(device)
    if device == 'cuda':
        net = torch.nn.DataParallel(net)
        cudnn.benchmark = True
    print(f'learning rate = {learning_rate}')

    result = train(net, LR=learning_rate)
    results[f'Learning Rate={learning_rate}'] = result
    
    learning_rate += 0.02

    save_to_excel(parse_results(results, 'validation'), 'WIP_LR_results_val')
    save_to_excel(parse_results(results, 'train'), 'WIP_LR_results_train')

save_to_excel(parse_results(results, 'validation'), 'LR_results_val')
save_to_excel(parse_results(results, 'train'), 'LR_results_train')